# Text Classification with AG's News Topics - Ludwig CLI

*We recommend using a GPU runtime for this example. In the Colab menu bar, choose **Runtime** > **Change Runtime Type** and choose **GPU** under Hardware Accelerator.*

In this notebook, we will show how to use the Ludwig CLI to:


1.   [Download a Dataset](#scrollTo=Download_Dataset)
2.   [Train a Ludwig Model](#scrollTo=Train)
3.   [Evaluate the trained model](#scrollTo=Evaluate)
4.   [Visualize training and test metrics](#scrollTo=Visualize_Metrics)
5.   [Make predictions on New Data](#scrollTo=Make_Predictions_on_New_Data)

In [1]:
# Prerequisite: Run this cell once to install the latest version of Ludwig in the Colab environment
!python -m pip install git+https://github.com/ludwig-ai/ludwig.git --quiet
!python -m pip install -U dask --quiet  # TODO: remove this and install ludwig release version

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 47 kB 3.4 MB/s 
     |████████████████████████████████| 134 kB 23.6 MB/s 
     |████████████████████████████████| 332 kB 38.8 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 144 kB 42.1 MB/s 
     |████████████████████████████████| 271 kB 48.9 MB/s 
     |████████████████████████████████| 1.1 MB 12.4 MB/s 
     |████████████████████████████████| 596 kB 35.3 MB/s 


# Download Dataset

We'll be using AG's news topic classification dataset, a common benchmark dataset for text classification. This dataset is a subset of the full AG news dataset, constructed by choosing 4 largest classes from the original corpus. Each class contains 30,000 training samples and 1,900 testing samples. The total number of training samples is 120,000 and testing 7,600.


This dataset contains three columns:

| column      | description                                                |
|-------------|------------------------------------------------------------|
| class_index | 1-4: "world", "sports", "business", "sci/tech" respectively |
| title       | Title of the news article                                  |
| description | Description of the news article                            |

In [2]:
# Downloads the AG news dataset to the current working directory.
!ludwig datasets download agnews

NumExpr defaulting to 2 threads.
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Datasets download



# Train

## Define ludwig config

The Ludwig config declares the machine learning task. It tells Ludwig what to predict, what columns to use as input, and optionally specifies the model type and hyperparameters.

Here, for simplicity, we'll try to predict **class_index** from **title**.

In [3]:
config_yaml = """
input_features:
  -
    name: title
    type: text
    level: word
    encoder: parallel_cnn
output_features:
  -
    name: class_index
    type: category
preprocessing:
  force_split: true
  split_probabilities: [0.7, 0.1, 0.2]
trainer:
  epochs: 3
"""

# Writes config to a file called "config.yaml"
with open("config.yaml", "w") as f:
  f.write(config_yaml)

## Create and train a model

In [4]:
# Trains the model. This cell might take a few minutes.
!ludwig train --dataset agnews.csv -c config.yaml

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Train


╒════════════════════════╕
│ EXPERIMENT DESCRIPTION │
╘════════════════════════╛

╒══════════════════╤═══════════════════════════════════════════════════════════════════╕
│ Experiment name  │ experiment                                                        │
├──────────────────┼───────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                               │
├──────────────────┼───────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/experiment_run                                   │
├──────────────────┼───────────────────────────────────────────────────────────────────┤
│ ludwig_vers

# Evaluate

In [5]:
# Generates predictions and performance statistics for the test set.
!ludwig evaluate --model_path results/experiment_run/model \
                 --dataset agnews.csv \
                 --split test \
                 --output_directory test_results

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Evaluate

Dataset path: agnews.csv
Model path: results/experiment_run/model

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)
Loading metadata from: results/experiment_run/model/training_set_metadata.json
Evaluation: 100% 199/199 [00:03<00:00, 62.60it/s]

===== class_index =====
accuracy: 0.8737768530845642
hits_at_k: 0.9891539216041565
loss: 0.37075480818748474
overall_stats: { 'avg_f1_score_macro': 0.8734849817505728,
  'avg_f1_score_micro': 0.873776869

# Visualize Metrics

In [9]:
# Visualizes confusion matrix, which gives an overview of classifier performance
# for each class.
!ludwig visualize --visualization confusion_matrix \
                  --ground_truth_metadata results/experiment_run/model/training_set_metadata.json \
                  --test_statistics test_results/test_statistics.json \
                  --output_directory visualizations \
                  --file_format png


# If you run ludwig visualize locally, visualizations will automatically show in
# a window. Here in Colab, we can run the following code to load and display
# generated plots inline.
from IPython import display
import ipywidgets
from pathlib import Path

ipywidgets.HBox([
  ipywidgets.Image(value=Path("visualizations/confusion_matrix__class_index_top5.png").read_bytes()),
  ipywidgets.Image(value=Path("visualizations/confusion_matrix_entropy__class_index_top5.png").read_bytes()),
])

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'


In [13]:
# Visualizes learning curves, which show how performance metrics changed over
# time during training.
!ludwig visualize --visualization learning_curves \
                  --ground_truth_metadata results/experiment_run/model/training_set_metadata.json \
                  --training_statistics results/experiment_run/training_statistics.json \
                  --file_format png \
                  --output_directory visualizations


# If you run ludwig visualize locally, visualizations will automatically show in
# a window. Here in Colab, we can run the following code to load and display
# generated plots inline.
from IPython import display
import ipywidgets
from pathlib import Path

ipywidgets.VBox([
  ipywidgets.HBox([
    ipywidgets.Image(value=Path("visualizations/learning_curves_combined_loss.png").read_bytes()),
    ipywidgets.Image(value=Path("visualizations/learning_curves_class_index_loss.png").read_bytes()),
  ]),
  ipywidgets.HBox([
    ipywidgets.Image(value=Path("visualizations/learning_curves_class_index_accuracy.png").read_bytes()),
    ipywidgets.Image(value=Path("visualizations/learning_curves_class_index_hits_at_k.png").read_bytes())
  ]),
])

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'


# Make Predictions on New Data

Lastly we'll show how to generate predictions for new data.

The following are some recent news headlines. Feel free to edit or add your own strings to text_to_predict to see how the newly trained model classifies them.

In [14]:
import pandas as pd

text_to_predict = pd.DataFrame({
  "title": [
    "Google may spur cloud cybersecurity M&A with $5.4B Mandiant buy",
    "Europe struggles to meet mounting needs of Ukraine's fleeing millions",
    "How the pandemic housing market spurred buyer's remorse across America",
  ]
})

text_to_predict.to_csv('text_to_predict.csv', index=False)

In [17]:
!ludwig predict --model_path results/experiment_run/model \
                --dataset text_to_predict.csv \
                --output_directory predictions

# Loads predictions
predictions = pd.read_parquet('predictions/predictions.parquet')
predictions

NumExpr defaulting to 2 threads.
import ray failed with exception: No module named 'ray'
███████████████████████
█ █ █ █  ▜█ █ █ █ █   █
█ █ █ █ █ █ █ █ █ █ ███
█ █   █ █ █ █ █ █ █ ▌ █
█ █████ █ █ █ █ █ █ █ █
█     █  ▟█     █ █   █
███████████████████████
ludwig v0.5rc2 - Predict

Dataset path: text_to_predict.csv
Model path: results/experiment_run/model

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/conv.py:298: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)
  self.padding, self.dilation, self.groups)
Loading metadata from: results/experiment_run/model/training_set_metadata.json
Prediction: 100% 1/1 [00:00<00:00, 129.22it/s]
Saved to: predictions


,class_index_predictions,class_index_probabilities,class_index_probability,class_index_probabilities_<UNK>,class_index_probabilities_4,class_index_probabilities_2,class_index_probabilities_1,class_index_probabilities_3
0,4,"[5.439335787338351e-11, 0.966296374797821, 2.6...",0.966296,5.439336e-11,0.966296,0.000026,0.001354,0.032323
1,1,"[1.1360067730947776e-07, 0.004469613078981638,...",0.991744,1.136007e-07,0.004470,0.001153,0.991744,0.002632
2,4,"[1.1652912235149415e-06, 0.5125769376754761, 0...",0.512577,1.165291e-06,0.512577,0.000545,0.045636,0.441242
